The contents of this course including lectures, labs, homework assignments, and exams have all been adapted from the [Data 8 course at University California Berkley](https://data.berkeley.edu/education/courses/data-8). Through their generosity and passion for undergraduate education, the Data 8 community at Berkley has opened their content and expertise for other universities to adapt in the name of undergraduate education.

In [ ]:
!pip install datascience
from datascience import *
import numpy as np

from IPython.display import display, Math, Latex

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Chapter 15: Prediction

An important aspect of data science is to find out what data can tell us about the future. What do data about climate and pollution say about temperatures a few decades from now? Based on a person’s internet profile, which websites are likely to interest them? How can a patient’s medical history be used to judge how well he or she will respond to a treatment?

Let's load the Francis Galton data set on family and children heights that we have seen before. The table *heights* contains data on the midparent height and child’s height (all in inches) for a population of 934 adult “children”. Recall that the midparent height is an average of the heights of the two parents.

In [ ]:
# Data on heights of parents and their adult children
original = Table.read_table('galton.csv')
heights = Table().with_columns(
    'MidParent', original.column('midparentHeight'),
    'Child', original.column('childHeight')
    )

In [ ]:
heights

In [ ]:
heights.scatter('MidParent')

A primary reason for studying the data was to be able to predict the adult height of a child born to parents who were similar to those in the dataset. We made these predictions in Section 8.1, after noticing the positive association between the two variables.

In [ ]:
def predict_child(mpht):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of mpht.
    
    The prediction is the average height of the children 
    whose midparent height is in the range mpht plus or minus 0.5 inches.
    """
    
    close_points = heights.where('MidParent', are.between(mpht-0.5, mpht + 0.5))
    return close_points.column('Child').mean()                       

In [ ]:
# Apply predict_child to all the midparent heights

heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
    )

In [ ]:
# Draw the original scatter plot along with the predicted values

heights_with_predictions.scatter('MidParent')

The prediction at a given midparent height lies roughly at the center of the vertical strip of points at the given height. This method of prediction is called regression. 

### Correlation
* In this section we will develop a measure of how tightly clustered a scatter diagram is about a straight line. Formally, this is called measuring linear association.

The table hybrid contains data on hybrid passenger cars sold in the United States from 1997 to 2013. The data were adapted from the online data archive of Prof. Larry Winner of the University of Florida. The columns:

   * vehicle: model of the car

   * year: year of manufacture

   * msrp: manufacturer’s suggested retail price in 2013 dollars

   * acceleration: acceleration rate in km per hour per second

   * mpg: fuel econonmy in miles per gallon

   * class: the model’s class.


In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.scatter('acceleration', 'msrp')

In [ ]:
hybrid.scatter('mpg', 'msrp')

The above plot appears to be non-linear

In [ ]:
# Selecting 
suv = hybrid.where('class', 'SUV')
suv.scatter('mpg', 'msrp')

Whene selecting only SUVs the data is more linearly correlated.

In [ ]:
suv.scatter('acceleration', 'msrp')

We can derive useful information from the general orientation and shape of a scatter diagram even without paying attention to the units in which the variables were measured.

We can plot all the variables in standard units and the plots would look the same. This gives us a way to compare the degree of linearity in two scatter diagrams.

Recall that in an earlier section we defined the function standard_units to convert an array of numbers to standard units.

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers))/np.std(any_numbers)  

In [ ]:
Table().with_columns(
    'mpg (standard units)',  standard_units(suv.column('mpg')), 
    'msrp (standard units)', standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

In [ ]:
Table().with_columns(
    'acceleration (standard units)', standard_units(suv.column('acceleration')), 
    'msrp (standard units)',         standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

We will now define a measure that uses standard units to quantify the kinds of association that we have seen.

### The Correlation Coefficient

* The correlation coefficient measures how clustered the scatter diagram is around a straight line.

The term correlation coefficient isn’t easy to say, so it is usually shortened to correlation and denoted by $r$.

Here are some mathematical facts about $r$ that we will just observe by simulation.

* The correlation coefficient is a number between $-1$ and $1$.

* $r$ measures the extent to which the scatter plot clusters around a straight line.

* $r=1$ if the scatter diagram is a perfect straight line sloping upwards, and $r=-1$ if the scatter diagram is a perfect straight line sloping downwards.

Formula for $r$ is the average of the products of the two variables, when both variables are measured in standard units.

Here are the steps in the calculation. We will apply the steps to a simple table of values of $x$ and $y$.

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
t = Table().with_columns(
        'x', x,
        'y', y
    )
t

Based on the scatter diagram, we expect that $r$ will be positive but not equal to 1.

In [ ]:
t.scatter(0, 1, s=30, color='red')

**Step 1.** Convert each variable to standard units.

In [ ]:
t_su = t.with_columns(
        'x (standard units)', standard_units(x),
        'y (standard units)', standard_units(y)
    )
t_su

**Step 2.** Multiply each pair of standard units.

In [ ]:
t_product = t_su.with_column('product of standard units', t_su.column(2) * t_su.column(3))
t_product

**Step 3.** is the average of the products computed in Step 2.

In [ ]:
# r is the average of the products of standard units

r = np.mean(t_product.column(4))
r

As expected, $r$ is positive but not equal to 1.

### Properties of $r$

* $r$ is a pure number. It has no units.

* $r$ is based on standard units.

* $r$ is unaffected by changing the units on either axis. 

* $r$ is unaffected by switching the axes. Algebraically, this is because the product of standard units does not depend on which variable is called $x$ and which $y$. Geometrically, switching axes reflects the scatter plot about the line $y=x$, but does not change the amount of clustering nor the sign of the association.

In [ ]:
t.scatter('y', 'x', s=30, color='red')

We are going to be calculating correlations repeatedly so lets define a function that computes it by performing all the steps described above.

In [ ]:
def correlation(t, x, y):
    return np.mean(standard_units(t.column(x))*standard_units(t.column(y)))

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
correlation(t, 'y', 'x')

In [ ]:
correlation(suv, 'mpg', 'msrp')

In [ ]:
correlation(suv, 'acceleration', 'msrp')

#### Association is not Causation¶
Correlation only measures association. Correlation does not imply causation. Though the correlation between the weight and the math ability of children in a school district may be positive, that does not mean that doing math makes children heavier or that putting on weight improves the children’s math skills. Age is a confounding variable: older children are both heavier and better at math than younger children, on average.

Correlation measures only one kind of association – linear. Variables that have strong non-linear association might have very low correlation. Here is an example of variables that have a perfect quadratic relation $y=x^2$ but have correlation equal to 0.

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

Outliers can have a big effect on correlation. Here is an example where a scatter plot for which $r=1$ is turned into a plot for which $r=0$, by the addition of just one outlying point.

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

### Ecological Correlations

Correlations based on aggregated data can be misleading. As an example, here are data on the Critical Reading and Math SAT scores in 2014. 

* All 50 states and Washington, D.C. 
* Participation Rate contains the percent of high school seniors who took the test


In [ ]:
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')

The data in the table have been created by lumping all the students in each state into a single point at the average values of the two variables in that state. But not all students in the state will be at that point, as students vary in their performance. If you plot a point for each student instead of just one for each state, there will be a cloud of points around each point in the figure above. The overall picture will be more fuzzy. The correlation between the Math and Critical Reading scores of the students will be lower than the value calculated based on state averages.

Correlations based on aggregates and averages are called ecological correlations and are frequently reported. As we have just seen, they must be interpreted with care.

In 2012, a [paper](https://www.biostat.jhsph.edu/courses/bio621/misc/Chocolate%20consumption%20cognitive%20function%20and%20nobel%20laurates%20(NEJM).pdf) in the respected New England Journal of Medicine examined the relation between chocolate consumption and Nobel Prizes in a group of countries. The following graph, provided in the paper, should motivate you to go and take a look.

<img src="Correlation_65_0.png" width=600 height=600 />

Let's go back to the parent/child height data

In [ ]:
heights_with_predictions.scatter('MidParent')

We can convert the data into standard units and create a plot that looks exaclty like the one above

In [ ]:
def standard_units(xyz):
    "Convert any array of numbers to standard units."
    return (xyz - np.mean(xyz))/np.std(xyz)  

In [ ]:
heights_SU = Table().with_columns(
    'MidParent SU', standard_units(heights.column('MidParent')),
    'Child SU', standard_units(heights.column('Child'))
)
heights_SU

In [ ]:
def predict_child_su(mpht_su):
    """Return a prediction of the height (in standard units) of a child 
    whose parents have a midparent height of mpht_su in standard units.
    """
    close = 0.5/sd_midparent
    close_points = heights_SU.where('MidParent SU', are.between(mpht_su-close, mpht_su + close))
    return close_points.column('Child SU').mean()   

In [ ]:
sd_midparent = np.std(heights.column(0))
sd_midparent

In [ ]:
heights_with_su_predictions = heights_SU.with_column(
    'Prediction SU', heights_SU.apply(predict_child_su, 'MidParent SU')
    )

In [ ]:
heights_with_su_predictions.scatter('MidParent SU')

### The Equation of the Regression Line

In regression, we use the value of one variable (which we will call $x$) to predict the value of another (which we will call $y$). When the variables and are measured in standard units, the regression line for predicting $y$ based on $x$ has slope $r$ and passes through the origin. Thus the equation of the regression line can be written as: $\mbox{estimate of }y ~=~ r \cdot x ~~~
\mbox{when both variables are measured in standard units} $

In the original units of the data, this becomes 

$ \frac{\mbox{estimate of}~y ~-~\mbox{average of}~y}{\mbox{SD of}~y} ~=~ r \times \frac{\mbox{the given}~x ~-~\mbox{average of}~x}{\mbox{SD of}~x}$

<img src="regline.png" width=600 height=600 />

The slope and intercept of the regression line in original units can be derived from the diagram above.

$\mathbf{\mbox{slope of the regression line}} ~=~ r \cdot
\frac{\mbox{SD of }y}{\mbox{SD of }x} $

$\mathbf{\mbox{intercept of the regression line}} ~=~
\mbox{average of }y ~-~ \mbox{slope} \cdot \mbox{average of }x
$

The three functions below compute the correlation, slope, and intercept.

In [ ]:
def correlation(t, label_x, label_y):
    return np.mean(standard_units(t.column(label_x))*standard_units(t.column(label_y)))

def slope(t, label_x, label_y):
    r = correlation(t, label_x, label_y)
    return r*np.std(t.column(label_y))/np.std(t.column(label_x))

def intercept(t, label_x, label_y):
    return np.mean(t.column(label_y)) - slope(t, label_x, label_y)*np.mean(t.column(label_x))

In [ ]:
family_r = correlation(heights, 'MidParent', 'Child')
family_r

In [ ]:
family_slope = slope(heights, 'MidParent', 'Child')
family_intercept = intercept(heights, 'MidParent', 'Child')
family_slope, family_intercept

The equation of the regression line is

$  \mbox{estimate of child's height} ~=~ 0.64 \cdot \mbox{midparent height} ~+~ 22.64 $

This is also known as the regression equation. The principal use of the regression equation is to predict $y$ based on $x$.

For example, for a midparent height of 70.48 inches, the regression equation predicts the child’s height to be 67.56 inches.

In [ ]:
family_slope * 70.48 + family_intercept

Our original prediction, created by taking the average height of all children who had midparent heights close to 70.48, came out to be pretty close: 67.63 inches compared to the regression line’s prediction of 67.55 inches.

In [ ]:
heights_with_predictions.where('MidParent', are.equal_to(70.48)).show(3)

In [ ]:
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', family_slope * heights.column('MidParent') + family_intercept
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('MidParent')

The blue dots show the regression predictions

The predictions all lie on the line and are known as the “fitted values”. The function ***fit*** takes the name of the table and the labels of $x$ and $y$, and returns an array of fitted values.

In [ ]:
def fit(table, x, y):
    """Return the height of the regression line at each x value."""
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * table.column(x) + b

In [ ]:
heights.with_column('Fitted', fit(heights, 'MidParent', 'Child')).scatter('MidParent')

Another way to draw the line is to use the option ***fit_line=True*** with the Table method ***scatter***.

In [ ]:
heights.scatter('MidParent', fit_line=True)

The slope is a ratio, and it worth taking a moment to study the units in which it is measured.

In [ ]:
slope(heights, 'MidParent', 'Child')

The slope of the regression line is about 0.64 child inches per midparent height in inches. This means that for parents whos mid-heights are 1 inch apart our prediction of child height will differ by about 0.64 inches.

### The Method of Least Squares

We will use the text “Little Women” to estimate the number of characters (that is, letters, spaces punctuation marks, and so on) based on the number of periods.

In [ ]:
little_women = Table.read_table('little_women.csv')
little_women = little_women.move_to_start('Periods')
little_women.show(3)

In [ ]:
little_women.scatter('Periods', 'Characters')

In [ ]:
correlation(little_women, 'Periods', 'Characters')

The scatter plot is remarkably close to linear, and the correlation is more than 0.92.

In [ ]:
lw_with_predictions = little_women.with_column('Linear Prediction', fit(little_women, 'Periods', 'Characters'))
lw_with_predictions.scatter('Periods')

Corresponding to each point on the scatter plot, there is an error of prediction calculated as the actual value minus the predicted value. It is the vertical distance between the point and the line, with a negative sign if the point is below the line.

In [ ]:
actual = lw_with_predictions.column('Characters')
predicted = lw_with_predictions.column('Linear Prediction')
errors = actual - predicted

In [ ]:
lw_with_predictions.with_column('Error', errors)

We can use slope and intercept to calculate the slope and intercept of the fitted line. The graphs below shows the line of fit for a few possible regression lines. The errors corresponding to four of the points are shown in red. There is nothing special about those four points. They were just chosen for clarity of the display. 

In [ ]:
lw_reg_slope = slope(little_women, 'Periods', 'Characters')
lw_reg_intercept = intercept(little_women, 'Periods', 'Characters')
print("slope:", lw_reg_slope,  "intercept:", lw_reg_intercept)

<img src="Method_of_Least_Squares_17_1.png" width=400 height=400 />

<img src="Method_of_Least_Squares_19_0.png" width=400 height=400 />

<img src="Method_of_Least_Squares_20_0.png" width=400 height=400 />

If you use any arbitrary line to calculate your estimates, then some of your errors are likely to be positive and others negative. To avoid cancellation when measuring the rough size of the errors, we will take the mean of the squared errors rather than the mean of the errors themselves.

The mean squared error of estimation is a measure of roughly how big the squared errors are, but as we have noted earlier, its units are hard to interpret. Taking the square root yields the root mean square error (rmse), which is in the same units as the variable being predicted and therefore much easier to understand.

* To get estimates of $y$ based on $x$, you can use any line you want.

* Every line has a root mean squared error of estimation.

* “Better” lines have smaller errors.

If we use the line

$\mbox{prediction} ~=~ ax + b$

it will have an mse that depends on the slope and the intercept . The function lw_mse takes the slope and intercept as its arguments and returns the corresponding mse.


In [ ]:
def lw_rmse(any_slope, any_intercept):
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    fitted = any_slope*x + any_intercept
    return np.mean((y - fitted) ** 2)**0.5

Let’s check that lw_mse gets the right answer for the root mean squared error of the regression line. 

In [ ]:
print("slope:", lw_reg_slope,  "intercept:", lw_reg_intercept)

In [ ]:
lw_rmse(lw_reg_slope, lw_reg_intercept)

In [ ]:
lw_rmse(-100, 50000)

In [ ]:
lw_rmse(90, 4000)

**The regression line is the unique straight line that minimizes the mean squared error of estimation among all straight lines.**

If we experiment with different values, we can find a low-error slope and intercept through trial and error, but that would take a while. Fortunately, there is a Python function that does all the trial and error for us.

The ***minimize*** function can be used to find the arguments of a function for which the function returns its minimum value. Python uses a similar trial-and-error approach, following the changes that lead to incrementally lower output values. 

The argument of minimize is a function that itself takes numerical arguments and returns a numerical value. For example, the function lw_mse takes a numerical slope and intercept as its arguments and returns the corresponding mse.

In [ ]:
best = minimize(lw_rmse)
best

In [ ]:
print("slope from formula:        ", lw_reg_slope)
print("slope from minimize:       ", best.item(0))
print("intercept from formula:    ", lw_reg_intercept)
print("intercept from minimize:   ", best.item(1))

We have found not only that the regression line minimizes root mean squared error, but also that minimizing mean squared error gives us the regression line. The regression line is the only line that minimizes mean squared error.

That is why the regression line is sometimes called the “least squares line.”

### Nonlinear Regression

Sometimes data may be curved and it is better to fit a curve than a straight line. The [study](https://digitalcommons.wku.edu/ijes/vol6/iss2/10/) postulated a quadratic relation between the weight lifted and the shot put distance. So let’s use quadratic functions as our predictors and see if we can find the best one.

We have to find the best quadratic function among all quadratic functions, instead of the best straight line among all straight lines. The method of least squares allows us to do this.

The mathematics of this minimization is complicated and not easy to see just by examining the scatter plot. But numerical minimization is just as easy as it was with linear predictors! We can get the best quadratic predictor by once again using minimize. Let’s see how this works.

Recall that a quadratic function has the form

$f(x) ~=~ ax^2 + bx + c$


for constants $a$, $b$, and $c$.

To find the best quadratic function to predict distance based on weight lifted, using the criterion of least squares, we will first write a function that takes the three constants as its arguments, calculates the fitted values by using the quadratic function above, and then returns the mean squared error.

The function is called shotput_quadratic_mse. Notice that the definition is analogous to that of lw_mse, except that the fitted values are based on a quadratic function instead of linear.

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput.show(3)

In [ ]:
def shotput_quadratic_mse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    fitted = a*(x**2) + b*x + c
    return np.mean((y - fitted) ** 2)**0.5

In [ ]:
best = minimize(shotput_quadratic_mse)
best

Our prediction of the shot put distance for an athlete who lifts $x$ kilograms is about

$-0.00104x^2 ~+~ 0.2827x - 1.5318$

meters. For example, if the athlete can lift 100 kilograms, the predicted distance is 16.33 meters. On the scatter plot, that’s near the center of a vertical strip around 100 kilograms.

In [ ]:
(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
x = shotput.column(0)
shotput_fit = best.item(0)*(x**2) + best.item(1)*x + best.item(2)

In [ ]:
shotput.with_column('Best Quadratic Curve', shotput_fit).scatter(0)

### Visual Diagnostics

Suppose a data scientist has decided to use linear regression to estimate values of one variable (called the response variable) based on another variable (called the predictor). To see how well this method of estimation performs, the data scientist must measure how far off the estimates are from the actual values. These differences are called residuals.

$\mbox{residual} ~=~ \mbox{observed value} ~-~ \mbox{regression estimate}$


Residuals are the vertical distances of the points from the regression line. There is one residual for each point in the scatter plot. 

$\mbox{residual} ~~ = ~~ y ~-~
\mbox{fitted value of }y
~~ = ~~ y ~-~
\mbox{height of regression line at }x$

The function residual calculates the residuals. The calculation assumes all the relevant functions we have already defined: standard_units, correlation, slope, intercept, and fit.

In [ ]:
def residual(table, x, y):
    return table.column(y) - fit(table, x, y)

In [ ]:
heights = heights.with_columns(
        'Fitted Value', fit(heights, 'MidParent', 'Child'),
        'Residual', residual(heights, 'MidParent', 'Child')
    )
heights

It is better to plot the data.

In [ ]:
def scatter_fit(table, x, y):
    table.scatter(x, y, s=15)
    plots.plot(table.column(x), fit(table, x, y), lw=4, color='gold')
    plots.xlabel(x)
    plots.ylabel(y)

In [ ]:
scatter_fit(heights, 'MidParent', 'Child')

In [ ]:
def residual_plot(table, x, y):
    x_array = table.column(x)
    t = Table().with_columns(
            x, x_array,
            'residuals', residual(table, x, y)
        )
    t.scatter(x, 'residuals', color='r')
    xlims = make_array(min(x_array), max(x_array))
    plots.plot(xlims, make_array(0, 0), color='darkblue', lw=4)
    plots.title('Residual Plot')

In [ ]:
residual_plot(heights, 'MidParent', 'Child')

The midparent heights are on the horizontal axis, as in the original scatter plot. But now the vertical axis shows the residuals. Notice that the plot appears to be centered around the horizontal line at the level 0 (shown in dark blue). Notice also that the plot shows no upward or downward trend. We will observe later that this lack of trend is true of all regressions.

**The residual plot of a good regression shows no pattern. The residuals look about the same, above and below the horizontal line at 0, across the range of the predictor variable.**

The function regression_diagnostic_plots draws the original scatter plot as well as the residual plot for ease of comparison.

In [ ]:
def regression_diagnostic_plots(table, x, y):
    scatter_fit(table, x, y)
    residual_plot(table, x, y)

In [ ]:
regression_diagnostic_plots(heights, 'MidParent', 'Child')

In [ ]:
regression_diagnostic_plots(shotput, 'Weight Lifted', 'Shot Put Distance')

When a residual plot shows a pattern or heteroscedasticity (uneven spread), there may be a non-linear relation between the variables.